### Middleware

In [16]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")
os.environ["GEMINI_API_KEY"]=os.getenv("GEMINI_API_KEY")

In [17]:
from langchain.agents import create_agent
from langchain.agents.middleware import SummarizationMiddleware
from langgraph.checkpoint.memory import InMemorySaver
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_google_genai import ChatGoogleGenerativeAI

###Message Summarization
model= ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.7)
agent=create_agent(
    model=model,
    checkpointer=InMemorySaver(),
    middleware=[
        SummarizationMiddleware(model=model,trigger=("messages",10),keep=("messages",4))
    ]
)

In [18]:
## Create a thread id

config={"configurable":{"thread_id":"test_1"}}

In [20]:
#Alternative test data

questions=[
    "What is 2+2 ?",
    "What is 2+10 ?",
    "What is 33/2 ?",
    "What is 2-2 ?",
    "What is 9*9 ?",
    "What is 7*7 ?",

]
for q in questions:
    response=agent.invoke({"messages": [HumanMessage(content=q)]},config)
    print(f"Messages: {response}")
    print(f"Messages: {len(response['messages'])}")


Messages: {'messages': [HumanMessage(content='Here is a summary of the conversation to date:\n\nThe user has asked several arithmetic questions, and the AI has consistently provided correct numerical answers. Examples of previous interactions include: 2+2=4, 2+10=12, 33/2=16.5.', additional_kwargs={}, response_metadata={}, id='f1a94622-3324-48b7-afe0-e95f00ae1839'), AIMessage(content='2 - 2 = 0', additional_kwargs={}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019b885e-3edc-73b1-ad99-5dfc64005c61-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 64, 'output_tokens': 7, 'total_tokens': 71, 'input_token_details': {'cache_read': 0}}), HumanMessage(content='What is 9*9 ?', additional_kwargs={}, response_metadata={}, id='2d687aa0-7ccf-4ded-b6d1-c556277a7b26'), AIMessage(content='9 * 9 = 81', additional_kwargs={}, response_metadata={'finish_reason': 'STOP', 'model_n

### Token size

In [26]:
from langchain.agents import create_agent
from langchain.agents.middleware import SummarizationMiddleware
from langgraph.checkpoint.memory import InMemorySaver
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.tools import tool 

@tool
def search_hotels(city:str)->str:
    """Search hotels - return long response to use more tokens"""
    return f""" Hotels in {city}:
    1.Grand Hotel -5 star , 1000/night , spa, gym, pool
    2.City Inn - 4 star . 900/night , business center
    3.Budget Stay - 3 star , 500/night , free wifi"""


# Message Summarization
model= ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.7)
agent=create_agent(
    model=model,
    checkpointer=InMemorySaver(),
    middleware=[
        SummarizationMiddleware(model=model,trigger=("tokens",550),keep=("tokens",200))
    ]
)
config={"configurable":{"thread_id":"test_1"}}

# Token counter(approximate)

def count_tokens(messages):
    total_chars= sum(len(str(m.content))for m in messages)
    return total_chars//4

In [27]:
# Run test

cities=["Paris", "London", "Tokyo", "Dubai", "Delhi"]

for city in cities:
    response=agent.invoke(
        {'messages': [HumanMessage(content=f"Find Hotels in {city}")]},
        config=config
    )

    tokens=count_tokens(response["messages"])
    print(f"{city}: ~{tokens} tokens, {len(response['messages'])} messages")
    print(f"{(response['messages'])}")

ChatGoogleGenerativeAIError: Error calling model 'gemini-2.5-flash' (RESOURCE_EXHAUSTED): 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. \n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 20, model: gemini-2.5-flash\nPlease retry in 51.886599827s.', 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Learn more about Gemini API quotas', 'url': 'https://ai.google.dev/gemini-api/docs/rate-limits'}]}, {'@type': 'type.googleapis.com/google.rpc.QuotaFailure', 'violations': [{'quotaMetric': 'generativelanguage.googleapis.com/generate_content_free_tier_requests', 'quotaId': 'GenerateRequestsPerDayPerProjectPerM

### Fractions

In [ ]:
from langchain.agents import create_agent
from langchain.agents.middleware import SummarizationMiddleware
from langgraph.checkpoint.memory import InMemorySaver
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.tools import tool 

@tool
def search_hotels(city:str)->str:
    """Search hotels - return long response to use more tokens"""
    return f""" Hotels in {city}:
    1.Grand Hotel -5 star , 1000/night , spa, gym, pool
    2.City Inn - 4 star . 900/night , business center
    3.Budget Stay - 3 star , 500/night , free wifi"""


# Message Summarization
model= ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.7)
agent=create_agent(
    model=model,
    checkpointer=InMemorySaver(),
    middleware=[
        SummarizationMiddleware(model=model,trigger=("fraction",0.005),keep=("fraction",0.002))
    ]
)
config={"configurable":{"thread_id":"test_1"}}

# Token counter(approximate)

def count_tokens(messages):
    total_chars= sum(len(str(m.content))for m in messages)
    return total_chars
cities=["Paris", "London", "Tokyo", "Dubai", "Delhi"]

for city in cities:
    response=agent.invoke(
        {'messages': [HumanMessage(content=f"Find Hotels in {city}")]},
        config=config
    )

    tokens=count_tokens(response["messages"])
    fractions=tokens/128000
    print(f"{city}: ~({fractions:.4%}) tokens, {len(response['messages'])} messages")
    print(f"{(response['messages'])}")

ChatGoogleGenerativeAIError: Error calling model 'gemini-2.5-flash' (RESOURCE_EXHAUSTED): 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. \n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 20, model: gemini-2.5-flash\nPlease retry in 34.80880245s.', 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Learn more about Gemini API quotas', 'url': 'https://ai.google.dev/gemini-api/docs/rate-limits'}]}, {'@type': 'type.googleapis.com/google.rpc.QuotaFailure', 'violations': [{'quotaMetric': 'generativelanguage.googleapis.com/generate_content_free_tier_requests', 'quotaId': 'GenerateRequestsPerDayPerProjectPerMo

### Human in the Loop

In [30]:
from langchain.agents import create_agent
from langchain.agents.middleware import HumanInTheLoopMiddleware
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.checkpoint.memory import InMemorySaver
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage, SystemMessage

def read_email_tool(email_id:str) ->str:
    """Mock function to read an email by its ID"""
    return f"Email content for ID: {email_id}'"

def send_email_tool(recipient:str, subject:str, body:str) -> str:
    """Mock function to sent an email"""
    return f"Email sent to {recipient} with subject '{subject}"


In [29]:
model= ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.7)
agent=create_agent(
    model=model,
    tools=[read_email_tool,send_email_tool],
    checkpointer=InMemorySaver(),
    middleware=[
        HumanInTheLoopMiddleware(interrupt_on={"send_email_tool":{"allowed_decisions":["approve","edit","reject"]},
                                               "read_email_tool": False}
                                )
    ]
)

In [33]:
from langchain_core.messages import HumanMessage, SystemMessage

config={"configurable":{"thread_id":"test-approve"}
        }

#Step 1: Request

result=agent.invoke(
    {"messages": [HumanMessage(content="Send email to john@test.com with subject 'Hello' and body 'How are you?")]},
    config=config
)

ChatGoogleGenerativeAIError: Error calling model 'gemini-2.5-flash' (RESOURCE_EXHAUSTED): 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. \n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 20, model: gemini-2.5-flash\nPlease retry in 53.179561076s.', 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Learn more about Gemini API quotas', 'url': 'https://ai.google.dev/gemini-api/docs/rate-limits'}]}, {'@type': 'type.googleapis.com/google.rpc.QuotaFailure', 'violations': [{'quotaMetric': 'generativelanguage.googleapis.com/generate_content_free_tier_requests', 'quotaId': 'GenerateRequestsPerDayPerProjectPerM

In [ ]:
result

In [32]:
from langgraph.types import Command

#Step 2:Approve

if '__interrupt__' in result:
    print("Pausing! Approved ....")

    result=agent.invoke(
        Command(
            resume={
                "decisions":[
                    {"type":"approve"}
                ]
            }
        ),
        config=config
    )
    print(f"Result:{result['messages'][-1].content}")
    

NameError: name 'result' is not defined